# Fashion MNIST using a custom training loop
we will build a custom training loop and a validation loop to traing a model on the Fashion mnist ds

https://github.com/sasidhar-programmer/Tensorflow_Advance_Techniques/blob/main/2-custom_and_distributed_training/week-2/C2_W2_Lab_2_training-categorical.ipynb

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tqdm import tqdm
import tensorflow_datasets as tfds
import matplotlib.ticker as mticker

ModuleNotFoundError: No module named 'tensorflow_datasets'